In [30]:
!pip install numpy
!pip install numpy-stl
!pip install tripy
!pip install plotly

In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from stl import mesh
# from google.colab import drive
import numpy as np

In [32]:


# drive.mount('/content/drive')
# drive_path = "/content/drive/MyDrive/Research/PhD/Scripts/"
# stl_file_path = drive_path + "test_part.stl"
stl_file_path = "./vessel_half.stl"

mesh_data = mesh.Mesh.from_file(stl_file_path)

# Extract vertices
vertices = mesh_data.vectors.reshape((-1, 3))

# Create a 3D scatter plot
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

fig.add_trace(go.Scatter3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    mode='markers',
    marker=dict(size=2),
))

# Set layout
fig.update_layout(scene=dict(aspectmode="data"))

# Show the plot
fig.show()

In [33]:
def on_top_surface(triangle, top_height):
  # print(triangle[:,2])
  return np.all(triangle[:,2]>top_height)

In [34]:
print(vertices.shape)
np.set_printoptions(suppress=True, precision=2)

height_tolerance = 1 #mm

# print(vertices)
# n_scores[:, 1:-1].max(axis=1)
# print("max z height", str(vertices[:,2].max()))

height_lb = vertices[:,2].max()-height_tolerance

triangles =  np.array([vertices[0:3,:]])

# organizes vertices into triangles
for i in range(int(vertices.shape[0]/3)):
  temp_tri = np.array([vertices[3*i:3*(i+1),:]])
  triangles = np.vstack((triangles, temp_tri))


# extracts top surface triangles
surface_tris = np.array([])
for ind in range(triangles.shape[0]):
  # checks if on top surface
  if(on_top_surface(triangles[ind], height_lb)):
    if(surface_tris.size == 0):
      surface_tris = np.array([triangles[ind]])
      print("surface triangle start shape", str(surface_tris.shape))
    else:
      surface_tris = np.vstack((surface_tris, [triangles[ind]]))

print(surface_tris)


(34464, 3)
surface triangle start shape (1, 3, 3)
[[[ 0.02 23.4   8.  ]
  [12.68 14.08  8.  ]
  [17.76 29.95  8.  ]]

 [[17.76 29.95  8.  ]
  [12.68 14.08  8.  ]
  [15.67 15.18  8.  ]]

 [[17.76 29.95  8.  ]
  [15.67 15.18  8.  ]
  [18.84 15.58  8.  ]]

 ...

 [[22.01 15.25  8.  ]
  [22.57 19.94  8.  ]
  [18.84 15.58  8.  ]]

 [[ 0.02 23.4   8.  ]
  [ 0.02  5.57  8.  ]
  [12.68 14.08  8.  ]]

 [[ 8.48 70.55  8.  ]
  [ 0.02 75.4   8.  ]
  [ 0.02 38.47  8.  ]]]


In [35]:
def shares_side(poly1,poly2):
  shared_vertices = 0
  # print("before squeeze")
  # print(poly1.shape)
  # print(poly2.shape)
  if(len(poly2.shape)>2):
    poly2 = np.squeeze(poly2)
  if(len(poly1.shape)>2):
    poly1 = np.squeeze(poly1)
  # print("after squeeze")
  # print(poly1.shape)
  # print(poly2.shape)
  # print("poly 1 shape", str(poly1.shape), "poly2 shape", str(poly2.shape))
  for i in range(poly1.shape[0]):
    for j in range(poly2.shape[0]):
      # print(str(poly1[i]), " =?", str(poly2[j]))
      # print(np.all(poly1[i] ==poly2[j]))
      if(np.all(poly1[i] ==poly2[j])):
        shared_vertices = shared_vertices+1
        if(shared_vertices >= 2):
          return True
  # if(shared_vertices>0):
    # print(str(shared_vertices), "shared vertices")
  return False

def combine(poly1,poly2):
    poly2 = np.squeeze(poly2)
    poly1 = np.squeeze(poly1)
    combined_poly = np.vstack((poly1,poly2))
    # print(combined_poly)
    return combined_poly


def combine_polygons(triangles):
    combined_polygons = [triangles[0]]
    # print(len(combined_polygons))
    # iterates through triangles
    for tri_ind in range(1,triangles.shape[0]):
      added_to_existing = False
      # checks combined polygon list
      for poly_ind in range(len(combined_polygons)):
        # checks for shared side
        # print("Checking ", str(poly_ind), "of ", str(len(combined_polygons)))
        # print("comparing ", str(triangles[tri_ind]), "to ", str(combined_polygons[poly_ind]))
        if(shares_side(triangles[tri_ind], combined_polygons[poly_ind])):
          combined_polygons[poly_ind] = combine(triangles[tri_ind], combined_polygons[poly_ind])
          added_to_existing = True
          # print("added  " , str(tri_ind), " to existing at", str(poly_ind))
          break
      if not added_to_existing:
          # combined_polygons = np.vstack(combined_polygons, triangles[tri_ind])

          combined_polygons.append(np.array([triangles[tri_ind]]))
          # print("added  " , str(tri_ind), " as new polygon")
          # print("Appended polygons")
          # print(combined_polygons)

      # print(combined_polygons)

    # print(len(combined_polygons))
    # print(combined_polygons)
    # print("----")
    return combined_polygons


def remove_repeat_from_polys(polygons):
  print(polygons)
  new_polygons = []

  for poly in polygons:
    # print(poly.shape)
    new_poly = poly[0]
    # print("New poly", str(new_poly))
    # print("---")
    for row in range(poly.shape[0]):
      if not shares_side(new_poly, np.array([poly[row,:]])):
        new_poly = combine(new_poly, poly[row,:])
        print("New Side added!")
        print(new_poly)
      else:
        print("Repeat side not added")
        print(new_poly)
    print("updated polygon:" )
    print(new_poly)
    new_polygons.append(new_poly)

  return new_polygons


def remove_repeat_from_polys2(polygons):
    unique_polygons = []

    for poly in polygons:
        is_unique = True
        for existing_poly in unique_polygons:
            if np.all(poly == existing_poly):
                is_unique = False
                break
        if is_unique:
            unique_polygons.append(poly)

    return unique_polygons


# print(surface_tris)
# polygons = combine_polygons(surface_tris)
# print("Before removal")
# for poly in polygons:
#   print(poly)
#   print('---')

# polygons = remove_repeat_from_polys(polygons)
# print("after removal")

# for poly in polygons:
#   print(poly)
#   print('---')

print(surface_tris)


[[[ 0.02 23.4   8.  ]
  [12.68 14.08  8.  ]
  [17.76 29.95  8.  ]]

 [[17.76 29.95  8.  ]
  [12.68 14.08  8.  ]
  [15.67 15.18  8.  ]]

 [[17.76 29.95  8.  ]
  [15.67 15.18  8.  ]
  [18.84 15.58  8.  ]]

 ...

 [[22.01 15.25  8.  ]
  [22.57 19.94  8.  ]
  [18.84 15.58  8.  ]]

 [[ 0.02 23.4   8.  ]
  [ 0.02  5.57  8.  ]
  [12.68 14.08  8.  ]]

 [[ 8.48 70.55  8.  ]
  [ 0.02 75.4   8.  ]
  [ 0.02 38.47  8.  ]]]


In [37]:
# exporting to a file
print(np.shape(surface_tris))

num_tris = surface_tris.shape[0]

with open("stand_in_environment.txt", "wb") as f:
	f.write(str("num triangles\n").encode())
	f.write(str(str(num_tris) + "\n\n").encode())
	for tri_ind in range(int(num_tris)):

		for point_ind in range(surface_tris[tri_ind].shape[0]):
			# f.write(str(str(surface_tris[tri_ind,point_ind, 0]), " ", str(surface_tris[tri_ind,point_ind, 1])).encode())
			np.savetxt(f,[surface_tris[tri_ind,point_ind,0:2]], fmt='%.2f',delimiter=',')
		f.write(b"\n")

(2537, 3, 3)
